In [32]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import urllib
import instaloader
from instaloader import *
from skimage import io
import os
from PIL import *
import requests
from io import BytesIO
import PIL.ImageStat
import pickle 

In [ ]:
#####Features######
#Number of hashtags in caption 
#Day 
#Time of Day 
### ^^^ Can get all of those directly from the post itself 
#Colors and their percentages
#Smile or not 
#Male Or Female 
#Clothes 
#Blur 
#Brightness
#Number of people in post 
#####NOT GENERALIZING WELL ON INSTAGRAM PICTURES FOR SMILE BECAUSE THE DOMAIN PICTURES ARE NOT AS FOCUSED AS
##THE TRAINING SET
###FOR EACH POST RUN SMILE DETECTION 100 Times. 
###RUN OBJECT DETECTION ON THE FILES, THE ONES WITH 1 person detected 
###HOW CENTRAL THE FACE IS TO THE PICTURE ?


In [2]:
L = instaloader.Instaloader()

In [3]:
# me = Profile.from_username(L.context, 'jamaalhay')
# profile = Profile.from_username(L.context, 'sabrinafacey')
# link = profile.get_posts()

In [3]:
def find_histogram(clt):
    """
    create a histogram with k clusters
    :param: clt
    :return:hist
    """
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    hist = hist.astype("float")
    hist /= hist.sum()

    return hist

In [4]:
def plot_colors2(hist, centroids):
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0

    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

In [5]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
    # return the image
    return image

In [7]:
post = load_structure_from_file(L.context, "Posts/424420030445644687.json")

In [17]:
post.date_local.year

2013

In [68]:
len(dict_tot)

21

In [66]:
#pic = url_to_image(post.url)
clt = KMeans(n_clusters=3)
dict_tot = {}
count = 0
for pot in newp:
    if count % 50 == 0:
        print(count)
    dictt = {}
    post = load_structure_from_file(L.context, "PostsNew/" + pot + "")
    try:
        image = io.imread(post.url)
    except:
        continue
    img = io.imread(post.url)
    img = img.reshape((img.shape[0] * img.shape[1],3)) #represent as row*column,channel number
    #cluster number
    clt.fit(img)
    dictt['col1'] = [hist[0], clt.cluster_centers_[0]]
    dictt['col2'] = [hist[1], clt.cluster_centers_[1]]
    dictt['col3'] = [hist[2], clt.cluster_centers_[2]]
    dict_tot[pot[:-5]] = dictt
    count += 1

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [19]:
def brightness(im_file):

    response = requests.get(im_file)
    img = Image.open(BytesIO(response.content)).convert('L')
    #im = Image.open(im_file).convert('L')
    stat = PIL.ImageStat.Stat(img)
    return stat.mean[0]

In [33]:
newp = os.listdir('PostsNew')
newps = []
newps.append(newp[0])

In [29]:
post = load_structure_from_file(L.context, "Posts/" + '1883110783414175586.json' + "")

In [30]:
post.owner_profile.followers

13792

In [47]:
#dictt = {}
count = 0
for pot in newp:
    if count % 500 == 0:
        print(count)
    post = load_structure_from_file(L.context, "Posts/" + pot + "")
    try:
        p = brightness(post.url)
    except:
        continue
    dictt[pot[:-5]] = p
    count +=1

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000


In [43]:
for key in dictt.keys():
    print(key)

1685887417998784488.


In [22]:
pots = os.listdir('Posts')

In [28]:
for i in pots: 
    try:
        post = load_structure_from_file(L.context, "Posts/" + str(i) + "")
    except:
        continue
    if int(post.date_local.year) >= 2018 and int(post.likes) > 100 and post.is_video == False and post.owner_profile.followers < 50000:
        save_structure_to_file(post, 'PostsNew/' + str(post.mediaid) + '.json')

In [48]:
print(len(dictt))

13263


In [49]:
pickle.dump( dictt, open( "save_brightness.p", "wb" ) )

In [50]:
favorite = pickle.load( open( "save_brightness.p", "rb" ) )

13263

In [53]:
def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [57]:
dictt = {}
count = 0
for pot in newp:
    if count % 100 == 0:
        print(count)
    # load the image, convert it to grayscale, and compute the
    # focus measure of the image using the Variance of Laplacian
    # method
    post = load_structure_from_file(L.context, "PostsNew/" + pot + "")
    try:
        image = io.imread(post.url)
    except:
        continue
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = variance_of_laplacian(gray)
    text = "Not Blurry"

    # if the focus measure is less than the supplied threshold,
    # then the image should be considered "blurry"
    if fm < 100:
        text = "Blurry"

    # show the image
    #cv2.imshow("Image", image)
    #key = cv2.waitKey(0)
    dictt[pot[:-5]] = text
    count += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200


In [58]:
pickle.dump( dictt, open( "save_blur.p", "wb" ) )